In [ ]:
import pandas as pd
import sys, os
sys.path.append(os.path.abspath(".."))

from typing import Union
from importlib import reload
import src.preprocessing
reload(src.preprocessing)
from src.preprocessing import compute_cancellation, aggregate_user_day_activity, add_days_since

In [2]:
root = '/Users/mdiaspinto/Documents/School/Python Data Science/Final Project/kaggle-churn'
df = pd.read_parquet(root + '/data/train.parquet')
df.head()

,status,gender,firstName,level,lastName,userId,ts,auth,page,sessionId,location,itemInSession,userAgent,method,length,song,artist,time,registration
0,200,M,Shlok,paid,Johnson,1749042,1538352001000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",278,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,524.32934,Ich mache einen Spiegel - Dream Part 4,Popol Vuh,2018-10-01 00:00:01,2018-08-08 13:22:21
992,200,M,Shlok,paid,Johnson,1749042,1538352525000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",279,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,178.02404,Monster (Album Version),Skillet,2018-10-01 00:08:45,2018-08-08 13:22:21
1360,200,M,Shlok,paid,Johnson,1749042,1538352703000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",280,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,232.61995,Seven Nation Army,The White Stripes,2018-10-01 00:11:43,2018-08-08 13:22:21
1825,200,M,Shlok,paid,Johnson,1749042,1538352935000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",281,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,265.50812,Under The Bridge (Album Version),Red Hot Chili Peppers,2018-10-01 00:15:35,2018-08-08 13:22:21
2366,200,M,Shlok,paid,Johnson,1749042,1538353200000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",282,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,471.69261,Circlesong 6,Bobby McFerrin,2018-10-01 00:20:00,2018-08-08 13:22:21


In [18]:
sample_time = df['time'].min() if 'time' in df.columns else '2023-01-01'
target_df = compute_cancellation(df, present_time=sample_time)
print('Computed cancellation target for', len(target_df), 'unique users')
print(f'Cancellations within 10 days of {sample_time}:')
print(target_df['cancellation_confirmed'].value_counts())

Computed cancellation target for 19140 unique users
Cancellations within 10 days of 2018-10-01 00:00:01:
cancellation_confirmed
0    18062
1     1078
Name: count, dtype: int64


In [8]:
# Check the structure of df
print("Columns:", df.columns.tolist())
print("\nShape:", df.shape)
print("\nFirst few rows:")
df.head()

Columns: ['status', 'gender', 'firstName', 'level', 'lastName', 'userId', 'ts', 'auth', 'page', 'sessionId', 'location', 'itemInSession', 'userAgent', 'method', 'length', 'song', 'artist', 'time', 'registration']

Shape: (17499636, 19)

First few rows:


,status,gender,firstName,level,lastName,userId,ts,auth,page,sessionId,location,itemInSession,userAgent,method,length,song,artist,time,registration
0,200,M,Shlok,paid,Johnson,1749042,1538352001000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",278,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,524.32934,Ich mache einen Spiegel - Dream Part 4,Popol Vuh,2018-10-01 00:00:01,2018-08-08 13:22:21
992,200,M,Shlok,paid,Johnson,1749042,1538352525000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",279,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,178.02404,Monster (Album Version),Skillet,2018-10-01 00:08:45,2018-08-08 13:22:21
1360,200,M,Shlok,paid,Johnson,1749042,1538352703000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",280,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,232.61995,Seven Nation Army,The White Stripes,2018-10-01 00:11:43,2018-08-08 13:22:21
1825,200,M,Shlok,paid,Johnson,1749042,1538352935000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",281,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,265.50812,Under The Bridge (Album Version),Red Hot Chili Peppers,2018-10-01 00:15:35,2018-08-08 13:22:21
2366,200,M,Shlok,paid,Johnson,1749042,1538353200000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",282,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,471.69261,Circlesong 6,Bobby McFerrin,2018-10-01 00:20:00,2018-08-08 13:22:21


In [9]:
# Check unique page values and time column
print("Unique page values:")
print(df['page'].unique())
print("\nPage value counts:")
print(df['page'].value_counts())
print("\nTime column dtype:", df['time'].dtype)
print("Sample time values:", df['time'].head())

Unique page values:
['NextSong' 'Downgrade' 'Help' 'Home' 'Thumbs Up' 'Add Friend'
 'Thumbs Down' 'Add to Playlist' 'Logout' 'About' 'Settings'
 'Save Settings' 'Cancel' 'Cancellation Confirmation' 'Submit Downgrade'
 'Roll Advert' 'Upgrade' 'Error' 'Submit Upgrade']

Page value counts:
page
NextSong                     14291433
Thumbs Up                      789391
Home                           645259
Add to Playlist                409606
Roll Advert                    284837
Add Friend                     262147
Logout                         204700
Thumbs Down                    164964
Downgrade                      124248
Settings                       101191
Help                            89035
Upgrade                         37696
About                           33117
Save Settings                   20370
Error                           17294
Submit Upgrade                  11381
Submit Downgrade                 4425
Cancellation Confirmation        4271
Cancel                 

In [ ]:
df_aggregated_new = aggregate_user_day_activity(df)

print(f"Original shape: {df.shape}")
print(f"Aggregated shape: {df_aggregated_new.shape}")
print(f"\nAggregated data columns: {df_aggregated_new.columns.tolist()}")
print(f"\nFirst few rows:")
df_aggregated_new.head(10)

Original shape: (17499636, 20)
Aggregated shape: (205676, 20)

Aggregated data columns: ['userId', 'date', 'About', 'Add Friend', 'Add to Playlist', 'Cancel', 'Downgrade', 'Error', 'Help', 'Home', 'Logout', 'NextSong', 'Roll Advert', 'Save Settings', 'Settings', 'Submit Downgrade', 'Submit Upgrade', 'Thumbs Down', 'Thumbs Up', 'Upgrade']

First few rows:


page,userId,date,About,Add Friend,Add to Playlist,Cancel,Downgrade,Error,Help,Home,Logout,NextSong,Roll Advert,Save Settings,Settings,Submit Downgrade,Submit Upgrade,Thumbs Down,Thumbs Up,Upgrade
0,1000025,2018-10-02,0,3,2,0,1,0,1,6,1,95,2,0,0,0,1,2,5,1
1,1000025,2018-10-03,0,6,13,0,2,0,2,13,2,317,1,0,1,0,0,3,21,0
2,1000025,2018-10-04,0,0,5,0,3,0,1,9,3,201,0,0,0,0,0,3,12,0
3,1000025,2018-10-05,0,0,1,0,2,0,0,2,0,22,0,0,0,0,0,0,0,0
4,1000025,2018-10-06,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0
5,1000025,2018-10-07,0,6,5,0,1,0,0,8,2,100,1,0,1,0,0,2,9,0
6,1000025,2018-10-09,0,0,0,0,1,0,0,2,1,29,0,0,0,0,0,0,3,0
7,1000025,2018-10-10,1,2,4,0,2,0,0,5,3,142,0,0,1,0,0,0,13,0
8,1000025,2018-10-11,0,1,3,0,2,0,1,8,1,115,0,1,1,0,0,0,7,0
9,1000025,2018-10-12,0,0,0,0,0,0,0,0,1,6,0,0,0,0,0,0,0,0


In [ ]:
# Add days_since columns for key events
df_aggregated_new = add_days_since(df_aggregated_new)

print("Added 'days_since' columns:")
print("- days_since_submit_downgrade")
print("- days_since_submit_upgrade")
print("- days_since_cancel")
print(f"\nFinal shape: {df_aggregated_new.shape}")
print(f"\nColumns: {df_aggregated_new.columns.tolist()}")
print(f"\nFirst few rows:")
df_aggregated_new.head()

Added 'days_since' columns:
- days_since_submit_downgrade
- days_since_submit_upgrade
- days_since_cancel

Final shape: (205676, 23)

Columns: ['userId', 'date', 'About', 'Add Friend', 'Add to Playlist', 'Cancel', 'Downgrade', 'Error', 'Help', 'Home', 'Logout', 'NextSong', 'Roll Advert', 'Save Settings', 'Settings', 'Submit Downgrade', 'Submit Upgrade', 'Thumbs Down', 'Thumbs Up', 'Upgrade', 'days_since_submit_downgrade', 'days_since_submit_upgrade', 'days_since_cancel']

First few rows:


page,userId,date,About,Add Friend,Add to Playlist,Cancel,Downgrade,Error,Help,Home,...,Save Settings,Settings,Submit Downgrade,Submit Upgrade,Thumbs Down,Thumbs Up,Upgrade,days_since_submit_downgrade,days_since_submit_upgrade,days_since_cancel
0,1000025,2018-10-02,0,3,2,0,1,0,1,6,...,0,0,0,1,2,5,1,None,0,None
1,1000025,2018-10-03,0,6,13,0,2,0,2,13,...,0,1,0,0,3,21,0,None,1,None
2,1000025,2018-10-04,0,0,5,0,3,0,1,9,...,0,0,0,0,3,12,0,None,2,None
3,1000025,2018-10-05,0,0,1,0,2,0,0,2,...,0,0,0,0,0,0,0,None,3,None
4,1000025,2018-10-06,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,None,4,None
